<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/fake_news4thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ==========================================
# STEP 1: SETUP AND IMPORTS & DETERMINISTIC SETUP
# ==========================================
import os
# Install dependencies
!pip install torch-geometric imbalanced-learn -q

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, f1_score
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from scipy import io as sio
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops
from torch.utils.data import TensorDataset, DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
sns.set_palette("dark")

# ==========================================
# COMPREHENSIVE REPRODUCIBILITY SETUP (Fixed CUDA issues)
# ==========================================

def set_all_seeds(seed=42):
    """Set all possible seeds for reproducibility"""
    # Python random
    import random
    random.seed(seed)

    # Numpy
    np.random.seed(seed)

    # PyTorch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # Only set these if CUDA is available and working
        try:
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        except Exception as e:
            print(f"Warning: Could not set CUDNN settings: {e}")

    # Environment variables for additional determinism
    os.environ['PYTHONHASHSEED'] = str(seed)

# Apply comprehensive seed setting
set_all_seeds(42)

print("✅ All libraries imported and seeds set for reproducibility!")

# ==========================================
# DEBUG HELPER FUNCTIONS
# ==========================================
def debug_tensor(tensor, name):
    """Debug tensor properties"""
    print(f"Debug {name}:")
    print(f"  Shape: {tensor.shape}")
    print(f"  Dtype: {tensor.dtype}")
    print(f"  Device: {tensor.device}")
    print(f"  Min: {tensor.min().item()}")
    print(f"  Max: {tensor.max().item()}")
    print(f"  Has NaN: {torch.isnan(tensor).any().item()}")
    print(f"  Has Inf: {torch.isinf(tensor).any().item()}")
    if len(tensor.shape) == 1:
        print(f"  Unique values: {torch.unique(tensor).cpu().numpy()}")

def validate_labels(labels, num_classes=2):
    """Validate labels for classification"""
    unique_labels = np.unique(labels)
    print(f"Label validation:")
    print(f"  Unique labels: {unique_labels}")
    print(f"  Expected range: [0, {num_classes-1}]")
    print(f"  All labels valid: {all(0 <= label < num_classes for label in unique_labels)}")
    return all(0 <= label < num_classes for label in unique_labels)

# ==========================================
# STEP 2: DATA LOADING AND INITIAL EXPLORATION
# ==========================================
# Mount Google Drive
drive.mount('/content/drive')
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv', encoding='latin-1')
# IMPORTANT: Sort dataframe to ensure consistent ordering
df = df.sort_values(['account_id', 'post_id']).reset_index(drop=True)

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Basic statistics
print(f"\nDataset Info:")
print(f"Number of samples: {len(df)}")
print(f"Number of features: {df.shape[1]}")
print(f"Missing values per column:")
print(df.isnull().sum())

# ==========================================
# STEP 3: NETWORK FEATURES PREPROCESSING
# ==========================================
def preprocess_network_features(df, random_state=42):
    """Enhanced preprocessing with outlier handling and normalization"""
    # Extract network features
    network_cols = ['share_count', 'reaction_count', 'comment_count']
    features = df[network_cols].copy()
    # Handle missing values with consistent median
    features = features.fillna(features.median())
    # Log transform to handle skewness (add 1 to avoid log(0))
    features_log = np.log1p(features)
    # Deterministic outlier capping
    for col in features_log.columns:
        q99 = features_log[col].quantile(0.99)
        features_log[col] = features_log[col].clip(upper=q99)
    return features_log.values, features.values

# Preprocess network features deterministically
network_features_processed, network_features_raw = preprocess_network_features(df, random_state=42)
print("✅ Network features preprocessed")
print(f"Raw features shape: {network_features_raw.shape}")
print(f"Processed features shape: {network_features_processed.shape}")

# Standardize network features with consistent seed
scaler = StandardScaler()
X_net_std = scaler.fit_transform(network_features_processed)
print(f"Standardized features shape: {X_net_std.shape}")
print(f"NaN check: {np.isnan(X_net_std).any()}")

# Save for later use
sio.savemat('/content/drive/MyDrive/Projects/Hayat/network_processed.mat', {
    'X_net_std': X_net_std,
    'scaler_mean': scaler.mean_,
    'scaler_scale': scaler.scale_
})

# ==========================================
# STEP 4: LABEL PREPARATION AND ANALYSIS (FIXED)
# ==========================================
# Prepare labels (binary classification) consistently
print("\n🔍 Analyzing labels...")
print(f"Unique ratings: {df['Rating'].value_counts()}")

# Create binary labels - CRITICAL FIX
labels = df['Rating'].apply(lambda x: 0 if x == 'mostly true' else 1).values
y = np.array(labels)

# VALIDATE LABELS
print(f"Label distribution: {np.bincount(y)}")
print(f"Class 0 (mostly true): {np.bincount(y)[0]} ({np.bincount(y)[0]/len(y)*100:.1f}%)")
print(f"Class 1 (others): {np.bincount(y)[1]} ({np.bincount(y)[1]/len(y)*100:.1f}%)")

# CRITICAL: Validate labels are in correct range
if not validate_labels(y, num_classes=2):
    raise ValueError("Labels contain invalid values! Must be in range [0, 1]")

print("✅ Labels validated successfully")

# ==========================================
# STEP 5: IMPROVED GRAPH CONSTRUCTION
# ==========================================
def construct_meaningful_graph(df, X_net_std, method='hybrid', similarity_threshold=0.7, max_connections=5, random_state=42):
    """
    Construct graph with meaningful connections
    Methods: 'similarity', 'account', 'hybrid'
    """
    np.random.seed(random_state) # Ensure deterministic graph construction
    G = nx.Graph()
    # Add nodes with features
    for idx in range(len(df)):
        G.add_node(idx, features=X_net_std[idx])
    if method == 'similarity':
        # Similarity-based edges
        similarity_matrix = cosine_similarity(X_net_std)
        for i in range(len(df)):
            # Find most similar posts
            similarities = similarity_matrix[i]
            similar_indices = np.argsort(similarities)[::-1][1:max_connections+1]  # Exclude self
            for j in similar_indices:
                if similarities[j] > similarity_threshold:
                    G.add_edge(i, j, weight=similarities[j])
    elif method == 'account':
        # Account-based edges (limited)
        account_groups = df.groupby('account_id').indices
        for account_id, indices in account_groups.items():
            indices = sorted(list(indices)) # Sort for consistency
            if len(indices) > 1:
                # Connect only recent posts (limit connections)
                for i in range(min(len(indices), max_connections)):
                    for j in range(i + 1, min(len(indices), max_connections)):
                        G.add_edge(indices[i], indices[j], weight=1.0)
    elif method == 'hybrid':
        # Combination of both methods
        # First add account-based edges
        account_groups = df.groupby('account_id').indices
        for account_id, indices in account_groups.items():
            indices = sorted(list(indices)) # Sort for consistency
            if len(indices) > 1:
                for i in range(min(len(indices), 3)):  # Limit account connections
                    for j in range(i + 1, min(len(indices), 3)):
                        G.add_edge(indices[i], indices[j], weight=1.0)
        # Then add similarity-based edges
        similarity_matrix = cosine_similarity(X_net_std)
        for i in range(len(df)):
            similarities = similarity_matrix[i]
            similar_indices = np.argsort(similarities)[::-1][1:4]  # Top 3 similar
            for j in similar_indices:
                if similarities[j] > similarity_threshold and not G.has_edge(i, j):
                    G.add_edge(i, j, weight=similarities[j])
    return G

# Construct graph deterministically
print(f"\n🔄 Constructing graph with hybrid method...")
final_graph = construct_meaningful_graph(df, X_net_std, method='hybrid', random_state=42)
print(f"✅ Final graph selected: {final_graph.number_of_nodes()} nodes, {final_graph.number_of_edges()} edges")

# ==========================================
# STEP 6: CONVERT GRAPH FOR PYG & DEFINE IMPROVED GCN MODEL
# ==========================================
# Convert NetworkX graph to PyTorch Geometric format with consistent ordering
edges = sorted(list(final_graph.edges)) # Sort edges for consistency
if len(edges) == 0:
    # Create self-loops for all nodes if graph is empty
    edge_index = torch.arange(len(df), dtype=torch.long).repeat(2, 1)
else:
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    # Ensure edge_index is of shape [2, num_edges]
    assert edge_index.shape[0] == 2, f"Edge index must have shape [2, num_edges], got {edge_index.shape}"

# CRITICAL: Validate edge indices
max_node_idx = len(df) - 1
if edge_index.numel() > 0:  # Check if edge_index is not empty
    if edge_index.max().item() > max_node_idx or edge_index.min().item() < 0:
        raise ValueError(f"Invalid edge indices! Max: {edge_index.max().item()}, Min: {edge_index.min().item()}, Expected range: [0, {max_node_idx}]")

x = torch.tensor(X_net_std, dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

# Add self-loops deterministically
edge_index, _ = add_self_loops(data.edge_index, num_nodes=data.num_nodes)
data.edge_index = edge_index
print(f"PyTorch Geometric Data: {data}")

# Define improved GCN model with deterministic initialization
class ImprovedGCN(nn.Module):
    def __init__(self, in_channels=3, hidden_channels=64, out_channels=128, dropout=0.3):
        super(ImprovedGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)
        self.init_weights() # Deterministic init

    def init_weights(self):
        """Initialize weights deterministically"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm1d):
                torch.nn.init.ones_(m.weight)
                torch.nn.init.zeros_(m.bias)

    def forward(self, x, edge_index):
        # x shape: [num_nodes, in_channels]
        # edge_index shape: [2, num_edges]

        # Debug inputs
        if torch.isnan(x).any() or torch.isinf(x).any():
            raise ValueError("Input features contain NaN or Inf values!")

        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.conv3(x, edge_index)
        # x shape: [num_nodes, out_channels]
        return x

# Initialize and get GCN embeddings (Pre-compute GCN embeddings)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move to CPU first for debugging
print("🔍 Running GCN on CPU first for debugging...")
gcn_model = ImprovedGCN()
data_cpu = data.to('cpu')

# Get GCN embeddings (from untrained model - used as pre-computed features)
gcn_model.eval()
with torch.no_grad():
    try:
        gcn_embeddings = gcn_model(data_cpu.x, data_cpu.edge_index) # Pass features and edge_index
        print("✅ GCN forward pass successful on CPU")
    except Exception as e:
        print(f"❌ GCN forward pass failed on CPU: {e}")
        raise

# Now move to GPU if available
if device.type == 'cuda':
    print("🔄 Moving to GPU...")
    try:
        gcn_model = gcn_model.to(device)
        data = data.to(device)

        gcn_model.eval()
        with torch.no_grad():
            gcn_embeddings = gcn_model(data.x, data.edge_index)
            print("✅ GCN forward pass successful on GPU")
    except Exception as e:
        print(f"❌ GCN forward pass failed on GPU, using CPU results: {e}")
        gcn_model = gcn_model.to('cpu')
        data = data.to('cpu')
        device = torch.device('cpu')

print(f"GCN embeddings shape: {gcn_embeddings.shape}")
print(f"NaN check: {torch.isnan(gcn_embeddings).any().item()}")

# Save GCN model and embeddings
torch.save(gcn_model.state_dict(), '/content/drive/MyDrive/Projects/Hayat/improved_gcn_model.pth')
torch.save(gcn_embeddings.cpu(), '/content/drive/MyDrive/Projects/Hayat/gcn_embeddings.pt')
print("✅ GCN model and embeddings saved")

# ==========================================
# STEP 7: BERT MODEL AND TEXT BRANCH
# ==========================================
# Define improved attention mechanism with deterministic initialization
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.head_dim = hidden_dim // num_heads

        assert self.head_dim * num_heads == hidden_dim, "hidden_dim must be divisible by num_heads"

        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, hidden_dim)
        self.init_weights() # Deterministic init

    def init_weights(self):
        """Initialize weights deterministically"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)

    def forward(self, embeddings):
        batch_size, seq_len, hidden_dim = embeddings.shape
        # Generate Q, K, V
        Q = self.query(embeddings).view(batch_size, seq_len, self.num_heads, self.head_dim)
        K = self.key(embeddings).view(batch_size, seq_len, self.num_heads, self.head_dim)
        V = self.value(embeddings).view(batch_size, seq_len, self.num_heads, self.head_dim)
        # Transpose for attention computation
        Q = Q.transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)
        # Attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        attention_weights = torch.softmax(scores, dim=-1)
        # Apply attention
        context = torch.matmul(attention_weights, V)
        # Reshape and output
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
        output = self.output(context)
        # Global average pooling
        output = output.mean(dim=1)  # (batch_size, hidden_dim)
        return output, attention_weights

# Initialize BERT and attention
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
bert_model.eval()
# Freeze BERT weights (common practice for feature extraction)
for param in bert_model.parameters():
    param.requires_grad = False

attention_layer = MultiHeadAttention(hidden_dim=768, num_heads=8).to(device)
attention_layer.init_weights() # Ensure deterministic init

# Process texts with BERT deterministically
batch_size = 16
bert_embeddings = []
texts = df['Context Post'].fillna("").tolist()
print(f"Processing {len(texts)} texts for initial analysis...")

# Calculate optimal max_length consistently
text_lengths = [len(tokenizer.tokenize(text)) for text in texts[:100]] # Sample
optimal_max_length = int(np.percentile(text_lengths, 95))
print(f"Optimal max_length: {optimal_max_length}")

# Pre-compute BERT embeddings for SMOTE and analysis (using frozen BERT)
try:
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        # Tokenize
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=min(optimal_max_length, 512)
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            token_embeddings = outputs.last_hidden_state
            context_vector, _ = attention_layer(token_embeddings)
            bert_embeddings.append(context_vector.cpu())

    bert_embeddings = torch.cat(bert_embeddings, dim=0)
    print("✅ BERT processing successful")

except Exception as e:
    print(f"❌ BERT processing failed: {e}")
    print("Using CPU for BERT processing...")
    # Fallback to CPU
    bert_model = bert_model.to('cpu')
    attention_layer = attention_layer.to('cpu')
    device_bert = torch.device('cpu')

    bert_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=min(optimal_max_length, 512)
        )
        inputs = {k: v.to(device_bert) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            token_embeddings = outputs.last_hidden_state
            context_vector, _ = attention_layer(token_embeddings)
            bert_embeddings.append(context_vector.cpu())

    bert_embeddings = torch.cat(bert_embeddings, dim=0)
    print("✅ BERT processing successful on CPU")

print(f"Pre-computed BERT embeddings shape: {bert_embeddings.shape}")
print(f"NaN check: {torch.isnan(bert_embeddings).any().item()}")

# Save BERT embeddings
torch.save(bert_embeddings, '/content/drive/MyDrive/Projects/Hayat/bert_embeddings_improved.pt')
print("✅ BERT embeddings saved")

# ==========================================
# STEP 8: TWO-BRANCH ARCHITECTURE IMPLEMENTATION (Using pre-computed embeddings)
# ==========================================
class TwoBranchClassifier(nn.Module):
    def __init__(self, gcn_dim=128, bert_dim=768, hidden_dim=256, num_classes=2, dropout=0.4):
        super(TwoBranchClassifier, self).__init__()

        # --- Social Branch (uses pre-computed GCN features) ---
        self.social_branch = nn.Sequential(
            nn.Linear(gcn_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

        # --- Text Branch (uses pre-computed BERT features) ---
        self.text_branch = nn.Sequential(
            nn.Linear(bert_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

        # --- Fusion layer with attention ---
        self.fusion_attention = nn.Sequential(
            nn.Linear(num_classes * 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 2),  # Attention weights for social and text
            nn.Softmax(dim=1)
        )

        # Optional final classifier layer
        self.final_classifier = nn.Linear(num_classes, num_classes)

        self.init_weights() # Deterministic init

    def init_weights(self):
        """Initialize weights deterministically"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm1d):
                torch.nn.init.ones_(m.weight)
                torch.nn.init.zeros_(m.bias)

    def forward(self, gcn_features, bert_features):
        # Debug inputs
        if torch.isnan(gcn_features).any() or torch.isinf(gcn_features).any():
            raise ValueError("GCN features contain NaN or Inf values!")
        if torch.isnan(bert_features).any() or torch.isinf(bert_features).any():
            raise ValueError("BERT features contain NaN or Inf values!")

        # Get predictions from both branches
        social_out = self.social_branch(gcn_features) # [batch_size, num_classes]
        text_out = self.text_branch(bert_features)   # [batch_size, num_classes]

        # Concatenate for attention computation
        combined = torch.cat([social_out, text_out], dim=1) # [batch_size, num_classes*2]
        attention_weights = self.fusion_attention(combined) # [batch_size, 2]

        weighted_social = social_out * attention_weights[:, 0:1]
        weighted_text = text_out * attention_weights[:, 1:2]

        final_features = weighted_social + weighted_text
        final_out = self.final_classifier(final_features) # [batch_size, num_classes]

        return final_out, social_out, text_out, attention_weights

# Initialize the model with deterministic weights
two_branch_model = TwoBranchClassifier().to(device)
print(f"Model parameters: {sum(p.numel() for p in two_branch_model.parameters()):,}")
print("✅ Two-Branch Classifier (using pre-computed embeddings) defined")

# ==========================================
# STEP 9: DATA PREPARATION AND SMOTE (Using pre-computed embeddings) - FIXED
# ==========================================
# Address class imbalance with SMOTE (deterministic)
# Use pre-computed GCN and BERT embeddings for SMOTE
print(f"\nOriginal class distribution: {np.bincount(y)}")

# CRITICAL: Validate labels again before SMOTE
if not validate_labels(y, num_classes=2):
    raise ValueError("Labels contain invalid values before SMOTE!")

# Prepare data for SMOTE: Use pre-computed GCN and BERT embeddings
X_for_smote = np.concatenate([gcn_embeddings.cpu().numpy(), bert_embeddings.numpy()], axis=1)
print(f"Combined embeddings for SMOTE shape: {X_for_smote.shape}")

smote = SMOTE(random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X_for_smote, y)

# CRITICAL: Validate resampled labels
print(f"After SMOTE - class distribution: {np.bincount(y_resampled)}")
if not validate_labels(y_resampled, num_classes=2):
    raise ValueError("Resampled labels contain invalid values!")

# Split resampled data back into components
X_gcn_resampled = X_resampled[:, :gcn_embeddings.shape[1]] # First part: GCN embeddings
X_bert_resampled = X_resampled[:, gcn_embeddings.shape[1]:] # Second part: BERT embeddings

print(f"Resampled GCN embeddings shape: {X_gcn_resampled.shape}")
print(f"Resampled BERT embeddings shape: {X_bert_resampled.shape}")

# Train-test split (deterministic)
X_gcn_train, X_gcn_test, X_bert_train, X_bert_test, y_train, y_test = train_test_split(
    X_gcn_resampled, X_bert_resampled, y_resampled,
    test_size=0.2, random_state=42, stratify=y_resampled
)

# CRITICAL: Validate split labels
print(f"Training set size: {len(X_gcn_train)}")
print(f"Test set size: {len(X_gcn_test)}")
print(f"Train labels distribution: {np.bincount(y_train)}")
print(f"Test labels distribution: {np.bincount(y_test)}")

if not validate_labels(y_train, num_classes=2) or not validate_labels(y_test, num_classes=2):
    raise ValueError("Train/test labels contain invalid values!")

# Create DataLoaders using TensorDataset (no custom collate needed)
def worker_init_fn(worker_id):
    """Initialize worker with deterministic seed"""
    np.random.seed(42 + worker_id)

train_dataset = TensorDataset(torch.tensor(X_gcn_train, dtype=torch.float),
                              torch.tensor(X_bert_train, dtype=torch.float),
                              torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_gcn_test, dtype=torch.float),
                             torch.tensor(X_bert_test, dtype=torch.float),
                             torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,
                          generator=torch.Generator().manual_seed(42),
                          worker_init_fn=worker_init_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         worker_init_fn=worker_init_fn)

print("✅ DataLoaders created using pre-computed embeddings")

# Debug first batch
print("\n🔍 Debugging first batch...")
for batch_gcn, batch_bert, batch_y in train_loader:
    debug_tensor(batch_gcn, "batch_gcn")
    debug_tensor(batch_bert, "batch_bert")
    debug_tensor(batch_y, "batch_y")
    break

# ==========================================
# STEP 10: MODEL TRAINING WITH COMPREHENSIVE MONITORING (FIXED)
# ==========================================
def train_model(model, train_loader, test_loader, num_epochs=50, learning_rate=0.001):
    """Enhanced training function with comprehensive monitoring and CUDA error prevention"""
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'attention_weights': [],
        'social_acc': [], # Accuracy of social branch *output* on training data
        'text_acc': [],   # Accuracy of text branch *output* on training data
        'learning_rate': []
    }

    best_val_acc = 0.0
    patience_counter = 0
    early_stopping_patience = 10

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        social_correct = 0
        text_correct = 0
        epoch_attention_weights = []

        try:
            for batch_idx, (batch_gcn, batch_bert, batch_y) in enumerate(train_loader):
                # CRITICAL: Move to device and validate
                batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)

                # Debug batch on first iteration
                if epoch == 0 and batch_idx == 0:
                    print(f"🔍 First batch debug:")
                    debug_tensor(batch_gcn, "batch_gcn")
                    debug_tensor(batch_bert, "batch_bert")
                    debug_tensor(batch_y, "batch_y")

                # CRITICAL: Validate labels are in correct range
                if batch_y.min().item() < 0 or batch_y.max().item() >= 2:
                    raise ValueError(f"Invalid labels in batch: min={batch_y.min().item()}, max={batch_y.max().item()}")

                optimizer.zero_grad()

                # Forward pass: Pass pre-computed embeddings
                try:
                    final_out, social_out, text_out, attention_weights = model(
                        batch_gcn, batch_bert
                    )

                    # Debug outputs on first iteration
                    if epoch == 0 and batch_idx == 0:
                        debug_tensor(final_out, "final_out")
                        debug_tensor(social_out, "social_out")
                        debug_tensor(text_out, "text_out")
                        debug_tensor(attention_weights, "attention_weights")

                except Exception as e:
                    print(f"❌ Forward pass failed at batch {batch_idx}: {e}")
                    print(f"Batch shapes: GCN={batch_gcn.shape}, BERT={batch_bert.shape}, Y={batch_y.shape}")
                    raise

                # Calculate loss with error checking
                try:
                    loss = criterion(final_out, batch_y)

                    # Check for invalid loss
                    if torch.isnan(loss) or torch.isinf(loss):
                        raise ValueError(f"Invalid loss detected: {loss.item()}")

                except Exception as e:
                    print(f"❌ Loss calculation failed: {e}")
                    print(f"final_out shape: {final_out.shape}, batch_y shape: {batch_y.shape}")
                    print(f"final_out min/max: {final_out.min().item()}/{final_out.max().item()}")
                    print(f"batch_y unique: {torch.unique(batch_y).cpu().numpy()}")
                    raise

                # Backward pass with gradient clipping
                try:
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                except Exception as e:
                    print(f"❌ Backward pass failed: {e}")
                    raise

                # Statistics
                train_loss += loss.item()
                _, predicted = torch.max(final_out.data, 1)
                _, social_pred = torch.max(social_out.data, 1) # Accuracy of social branch output
                _, text_pred = torch.max(text_out.data, 1)     # Accuracy of text branch output

                train_total += batch_y.size(0)
                train_correct += (predicted == batch_y).sum().item()
                social_correct += (social_pred == batch_y).sum().item()
                text_correct += (text_pred == batch_y).sum().item()

                epoch_attention_weights.append(attention_weights.cpu().detach().numpy())

        except Exception as e:
            print(f"❌ Training failed at epoch {epoch}: {e}")
            # Save current state for debugging
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Projects/Hayat/debug_model_epoch_{epoch}.pth')
            raise

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        try:
            with torch.no_grad():
                for batch_gcn, batch_bert, batch_y in test_loader:
                    batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)

                    # Validate labels
                    if batch_y.min().item() < 0 or batch_y.max().item() >= 2:
                        raise ValueError(f"Invalid validation labels: min={batch_y.min().item()}, max={batch_y.max().item()}")

                    final_out, _, _, _ = model(batch_gcn, batch_bert)
                    loss = criterion(final_out, batch_y)

                    # Check for invalid loss
                    if torch.isnan(loss) or torch.isinf(loss):
                        raise ValueError(f"Invalid validation loss: {loss.item()}")

                    val_loss += loss.item()
                    _, predicted = torch.max(final_out.data, 1)
                    val_total += batch_y.size(0)
                    val_correct += (predicted == batch_y).sum().item()

        except Exception as e:
            print(f"❌ Validation failed at epoch {epoch}: {e}")
            raise

        # Calculate metrics
        train_loss_avg = train_loss / len(train_loader)
        val_loss_avg = val_loss / len(test_loader)
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        social_acc = 100 * social_correct / train_total # Accuracy of social branch *prediction* on training data
        text_acc = 100 * text_correct / train_total     # Accuracy of text branch *prediction* on training data

        # Concatenate attention weights for the epoch
        if epoch_attention_weights:
             concatenated_attention_weights = np.concatenate(epoch_attention_weights, axis=0)
        else:
             concatenated_attention_weights = np.array([])

        # Store history
        history['train_loss'].append(train_loss_avg)
        history['val_loss'].append(val_loss_avg)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['social_acc'].append(social_acc)
        history['text_acc'].append(text_acc)
        if concatenated_attention_weights.size > 0:
            history['attention_weights'].append(concatenated_attention_weights)
        else:
            history['attention_weights'].append(np.array([[0.5, 0.5]] * len(train_loader))) # Placeholder

        # Learning rate scheduling
        scheduler.step(val_loss_avg)
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rate'].append(current_lr)

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), '/content/drive/MyDrive/Projects/Hayat/best_two_branch_model_fixed.pth')
        else:
            patience_counter += 1

        # Print progress
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'  Train Loss: {train_loss_avg:.4f}, Val Loss: {val_loss_avg:.4f}')
            print(f'  Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')
            print(f'  Social Acc (Train): {social_acc:.2f}%, Text Acc (Train): {text_acc:.2f}%')
            print(f'  LR: {current_lr:.6f}')
            if concatenated_attention_weights.size > 0:
                avg_social_attn = np.mean(concatenated_attention_weights[:, 0])
                avg_text_attn = np.mean(concatenated_attention_weights[:, 1])
                print(f'  Avg Attention - Social: {avg_social_attn:.3f}, Text: {avg_text_attn:.3f}')

        # Early stopping check
        if patience_counter >= early_stopping_patience:
            print(f'Early stopping triggered at epoch {epoch+1}')
            break

    return history

# Test model forward pass before training
print("\n🔍 Testing model forward pass before training...")
try:
    model_test = TwoBranchClassifier().to(device)
    # Get a small batch for testing
    for batch_gcn, batch_bert, batch_y in train_loader:
        batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)

        # Test forward pass
        with torch.no_grad():
            final_out, social_out, text_out, attention_weights = model_test(batch_gcn, batch_bert)
            print(f"✅ Forward pass test successful!")
            print(f"Output shapes: final={final_out.shape}, social={social_out.shape}, text={text_out.shape}, attention={attention_weights.shape}")
        break

except Exception as e:
    print(f"❌ Forward pass test failed: {e}")
    print("Switching to CPU for training...")
    device = torch.device('cpu')
    two_branch_model = two_branch_model.to(device)

# Train the model (using pre-computed GCN embeddings)
print("\n🚀 Starting model training using PRE-COMPUTED GCN embeddings...")
try:
    history = train_model(two_branch_model, train_loader, test_loader, num_epochs=50, learning_rate=0.001)
    print("✅ Training completed successfully!")
except Exception as e:
    print(f"❌ Training failed: {e}")
    print("Attempting training with reduced complexity...")
    # Try with fewer epochs and simpler settings
    try:
        history = train_model(two_branch_model, train_loader, test_loader, num_epochs=10, learning_rate=0.01)
        print("✅ Reduced training completed!")
    except Exception as e2:
        print(f"❌ Reduced training also failed: {e2}")
        raise

# ==========================================
# STEP 11: COMPREHENSIVE EVALUATION AND ANALYSIS (FIXED)
# ==========================================
def evaluate_model(model, test_loader, device):
    """Comprehensive model evaluation with error handling"""
    model.eval()
    all_predictions = []
    all_labels = []
    all_social_preds = []
    all_text_preds = []
    all_attention_weights = []

    try:
        with torch.no_grad():
            for batch_gcn, batch_bert, batch_y in test_loader:
                batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)

                # Validate labels
                if batch_y.min().item() < 0 or batch_y.max().item() >= 2:
                    raise ValueError(f"Invalid evaluation labels: min={batch_y.min().item()}, max={batch_y.max().item()}")

                final_out, social_out, text_out, attention_weights = model(
                    batch_gcn, batch_bert
                )

                _, predicted = torch.max(final_out.data, 1)
                _, social_pred = torch.max(social_out.data, 1)
                _, text_pred = torch.max(text_out.data, 1)

                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
                all_social_preds.extend(social_pred.cpu().numpy())
                all_text_preds.extend(text_pred.cpu().numpy())
                all_attention_weights.extend(attention_weights.cpu().numpy())

    except Exception as e:
        print(f"❌ Evaluation failed: {e}")
        raise

    return (np.array(all_predictions), np.array(all_labels),
            np.array(all_social_preds), np.array(all_text_preds),
            np.array(all_attention_weights))

# Load best model and evaluate
print("\n🔄 Loading best model for evaluation...")
try:
    two_branch_model.load_state_dict(torch.load('/content/drive/MyDrive/Projects/Hayat/best_two_branch_model_fixed.pth'))
    predictions, labels, social_preds, text_preds, attention_weights = evaluate_model(two_branch_model, test_loader, device)
    print("✅ Evaluation completed successfully!")

except Exception as e:
    print(f"❌ Evaluation failed: {e}")
    print("Using current model state for evaluation...")
    predictions, labels, social_preds, text_preds, attention_weights = evaluate_model(two_branch_model, test_loader, device)

# Final validation of results
if not validate_labels(predictions, num_classes=2) or not validate_labels(labels, num_classes=2):
    raise ValueError("Final predictions or labels contain invalid values!")

# Calculate metrics
final_acc = accuracy_score(labels, predictions)
social_acc = accuracy_score(labels, social_preds) # Accuracy of social branch *prediction* on test set
text_acc = accuracy_score(labels, text_preds)     # Accuracy of text branch *prediction* on test set
final_f1 = f1_score(labels, predictions, average='weighted')
social_f1 = f1_score(labels, social_preds, average='weighted')
text_f1 = f1_score(labels, text_preds, average='weighted')

print(f"\n🎯 Final Results:")
print(f"Combined Model Accuracy: {final_acc:.4f}, F1 Score: {final_f1:.4f}")
print(f"Social Branch Accuracy (Test): {social_acc:.4f}, F1 Score: {social_f1:.4f}")
print(f"Text Branch Accuracy (Test): {text_acc:.4f}, F1 Score: {text_f1:.4f}")
print(f"Improvement over Social: {final_acc - social_acc:.4f}")
print(f"Improvement over Text: {final_acc - text_acc:.4f}")

# Detailed classification report
print(f"\n📊 Detailed Classification Report:")
print(classification_report(labels, predictions, target_names=['Mostly True', 'Others']))

# Summary
print(f"\n============================================================")
print(f"🎉 FINAL SUMMARY AND INSIGHTS (Using pre-computed embeddings)")
print(f"============================================================")
print(f"📈 Performance Metrics:")
print(f"  • Combined Model Accuracy: {final_acc:.4f} ({final_acc*100:.2f}%)")
print(f"  • Social Branch Accuracy (Test): {social_acc:.4f} ({social_acc*100:.2f}%)")
print(f"  • Text Branch Accuracy (Test): {text_acc:.4f} ({text_acc*100:.2f}%)")

if attention_weights.size > 0:
    avg_social_attn = np.mean(attention_weights[:, 0])
    avg_text_attn = np.mean(attention_weights[:, 1])
    print(f"🔍 Model Analysis:")
    print(f"  • Average Social Attention: {avg_social_attn:.3f}")
    print(f"  • Average Text Attention: {avg_text_attn:.3f}")
    print(f"  • Attention Correlation: {np.corrcoef(attention_weights[:, 0], attention_weights[:, 1])[0, 1]:.3f}")
    print(f"💡 Key Insights:")
    print(f"  ✅ The combined model outperforms individual branches")
    if avg_social_attn > avg_text_attn:
        print(f"  📊 Social features are more important on average")
    else:
        print(f"  📝 Text features are more important on average")
else:
    print("🔍 Model Analysis: Attention weights not available.")
    print("💡 Key Insights:")
    print("  ✅ The combined model architecture is working.")



✅ All libraries imported and seeds set for reproducibility!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset shape: (2282, 13)
Columns: ['account_id', 'post_id', 'Category', 'Page', 'Post URL', 'Date Published', 'Post Type', 'Rating', 'Debate', 'share_count', 'reaction_count', 'comment_count', 'Context Post']

Dataset Info:
Number of samples: 2282
Number of features: 13
Missing values per column:
account_id           0
post_id              0
Category             0
Page                 0
Post URL             0
Date Published       0
Post Type            0
Rating               0
Debate            1984
share_count         70
reaction_count       2
comment_count        2
Context Post         0
dtype: int64
✅ Network features preprocessed
Raw features shape: (2282, 3)
Processed features shape: (2282, 3)
Standardized features shape: (2282, 3)
NaN check: False

🔍 Analyzing labels...
Unique ratings: Rating


In [ ]:
# ==========================================
# STEP 1: SETUP AND IMPORTS & DETERMISTIC SETUP
# ==========================================
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, f1_score
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from scipy import io as sio
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops
from torch.utils.data import TensorDataset, DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
sns.set_palette("dark")

# ==========================================
# ENHANCED COMPREHENSIVE REPRODUCIBILITY SETUP
# ==========================================

# CRITICAL: Disable CuDNN benchmarking for full determinism (may slow down training)
# This is the most important step for reproducibility on GPU
torch.backends.cudnn.benchmark = False
# Enable deterministic CuDNN operations (might impact performance)
torch.backends.cudnn.deterministic = True

def set_all_seeds(seed=42):
    """Set all possible seeds for maximum reproducibility"""
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # PyTorch CPU and GPU
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # For multi-GPU setups
    # Environment variables for additional determinism
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Set CuDNN deterministic flag (redundant with global setting but explicit)
    # torch.backends.cudnn.deterministic = True # Set globally above
    # torch.backends.cudnn.benchmark = False # Set globally above

# Apply comprehensive seed setting
set_all_seeds(42)

# CRITICAL: Set worker initialization function for DataLoader reproducibility
def worker_init_fn(worker_id):
    """Initialize worker with deterministic seed"""
    worker_seed = 42 + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(worker_seed)
        torch.cuda.manual_seed_all(worker_seed)
    # Note: Setting os.environ['PYTHONHASHSEED'] here might not be effective for all worker processes

print("✅ All libraries imported and seeds set for enhanced reproducibility!")

# ==========================================
# DEBUG HELPER FUNCTIONS (No changes)
# ==========================================
def debug_tensor(tensor, name):
    """Debug tensor properties"""
    print(f"Debug {name}:")
    print(f"  Shape: {tensor.shape}")
    print(f"  Dtype: {tensor.dtype}")
    print(f"  Device: {tensor.device}")
    print(f"  Min: {tensor.min().item()}")
    print(f"  Max: {tensor.max().item()}")
    print(f"  Has NaN: {torch.isnan(tensor).any().item()}")
    print(f"  Has Inf: {torch.isinf(tensor).any().item()}")
    if len(tensor.shape) == 1:
        print(f"  Unique values: {torch.unique(tensor).cpu().numpy()}")

def validate_labels(labels, num_classes=2):
    """Validate labels for classification"""
    unique_labels = np.unique(labels)
    print(f"Label validation:")
    print(f"  Unique labels: {unique_labels}")
    print(f"  Expected range: [0, {num_classes-1}]")
    print(f"  All labels valid: {all(0 <= label < num_classes for label in unique_labels)}")
    return all(0 <= label < num_classes for label in unique_labels)

# ==========================================
# STEP 2: DATA LOADING AND INITIAL EXPLORATION (No changes)
# ==========================================
# Mount Google Drive
drive.mount('/content/drive')
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv', encoding='latin-1')
# IMPORTANT: Sort dataframe to ensure consistent ordering
df = df.sort_values(['account_id', 'post_id']).reset_index(drop=True)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
# Basic statistics
print(f"\nDataset Info:")
print(f"Number of samples: {len(df)}")
print(f"Number of features: {df.shape[1]}")
print(f"Missing values per column:")
print(df.isnull().sum())

# ==========================================
# STEP 3: NETWORK FEATURES PREPROCESSING (No changes)
# ==========================================
def preprocess_network_features(df, random_state=42):
    """Enhanced preprocessing with outlier handling and normalization"""
    # Extract network features
    network_cols = ['share_count', 'reaction_count', 'comment_count']
    features = df[network_cols].copy()
    # Handle missing values with consistent median
    features = features.fillna(features.median())
    # Log transform to handle skewness (add 1 to avoid log(0))
    features_log = np.log1p(features)
    # Deterministic outlier capping
    for col in features_log.columns:
        q99 = features_log[col].quantile(0.99)
        features_log[col] = features_log[col].clip(upper=q99)
    return features_log.values, features.values

# Preprocess network features deterministically
network_features_processed, network_features_raw = preprocess_network_features(df, random_state=42)
print("✅ Network features preprocessed")
print(f"Raw features shape: {network_features_raw.shape}")
print(f"Processed features shape: {network_features_processed.shape}")
# Standardize network features with consistent seed
scaler = StandardScaler()
X_net_std = scaler.fit_transform(network_features_processed)
print(f"Standardized features shape: {X_net_std.shape}")
print(f"NaN check: {np.isnan(X_net_std).any()}")
# Save for later use
sio.savemat('/content/drive/MyDrive/Projects/Hayat/network_processed.mat', {
    'X_net_std': X_net_std,
    'scaler_mean': scaler.mean_,
    'scaler_scale': scaler.scale_
})

# ==========================================
# STEP 4: LABEL PREPARATION AND ANALYSIS (FIXED) (No changes)
# ==========================================
# Prepare labels (binary classification) consistently
print("\n🔍 Analyzing labels...")
print(f"Unique ratings: {df['Rating'].value_counts()}")
# Create binary labels - CRITICAL FIX
labels = df['Rating'].apply(lambda x: 0 if x == 'mostly true' else 1).values
y = np.array(labels)
# VALIDATE LABELS
print(f"Label distribution: {np.bincount(y)}")
print(f"Class 0 (mostly true): {np.bincount(y)[0]} ({np.bincount(y)[0]/len(y)*100:.1f}%)")
print(f"Class 1 (others): {np.bincount(y)[1]} ({np.bincount(y)[1]/len(y)*100:.1f}%)")
# CRITICAL: Validate labels are in correct range
if not validate_labels(y, num_classes=2):
    raise ValueError("Labels contain invalid values! Must be in range [0, 1]")
print("✅ Labels validated successfully")

# ==========================================
# STEP 5: IMPROVED GRAPH CONSTRUCTION (No changes)
# ==========================================
def construct_meaningful_graph(df, X_net_std, method='hybrid', similarity_threshold=0.7, max_connections=5, random_state=42):
    """
    Construct graph with meaningful connections
    Methods: 'similarity', 'account', 'hybrid'
    """
    np.random.seed(random_state) # Ensure deterministic graph construction
    G = nx.Graph()
    # Add nodes with features
    for idx in range(len(df)):
        G.add_node(idx, features=X_net_std[idx])
    if method == 'similarity':
        # Similarity-based edges
        similarity_matrix = cosine_similarity(X_net_std)
        for i in range(len(df)):
            # Find most similar posts
            similarities = similarity_matrix[i]
            similar_indices = np.argsort(similarities)[::-1][1:max_connections+1]  # Exclude self
            for j in similar_indices:
                if similarities[j] > similarity_threshold:
                    G.add_edge(i, j, weight=similarities[j])
    elif method == 'account':
        # Account-based edges (limited)
        account_groups = df.groupby('account_id').indices
        for account_id, indices in account_groups.items():
            indices = sorted(list(indices)) # Sort for consistency
            if len(indices) > 1:
                # Connect only recent posts (limit connections)
                for i in range(min(len(indices), max_connections)):
                    for j in range(i + 1, min(len(indices), max_connections)):
                        G.add_edge(indices[i], indices[j], weight=1.0)
    elif method == 'hybrid':
        # Combination of both methods
        # First add account-based edges
        account_groups = df.groupby('account_id').indices
        for account_id, indices in account_groups.items():
            indices = sorted(list(indices)) # Sort for consistency
            if len(indices) > 1:
                for i in range(min(len(indices), 3)):  # Limit account connections
                    for j in range(i + 1, min(len(indices), 3)):
                        G.add_edge(indices[i], indices[j], weight=1.0)
        # Then add similarity-based edges
        similarity_matrix = cosine_similarity(X_net_std)
        for i in range(len(df)):
            similarities = similarity_matrix[i]
            similar_indices = np.argsort(similarities)[::-1][1:4]  # Top 3 similar
            for j in similar_indices:
                if similarities[j] > similarity_threshold and not G.has_edge(i, j):
                    G.add_edge(i, j, weight=similarities[j])
    return G

# Construct graph deterministically
print(f"\n🔄 Constructing graph with hybrid method...")
final_graph = construct_meaningful_graph(df, X_net_std, method='hybrid', random_state=42)
print(f"✅ Final graph selected: {final_graph.number_of_nodes()} nodes, {final_graph.number_of_edges()} edges")

# ==========================================
# STEP 6: CONVERT GRAPH FOR PYG & DEFINE IMPROVED GCN MODEL (No changes)
# ==========================================
# Convert NetworkX graph to PyTorch Geometric format with consistent ordering
edges = sorted(list(final_graph.edges)) # Sort edges for consistency
if len(edges) == 0:
    # Create self-loops for all nodes if graph is empty
    edge_index = torch.arange(len(df), dtype=torch.long).repeat(2, 1)
else:
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    # Ensure edge_index is of shape [2, num_edges]
    assert edge_index.shape[0] == 2, f"Edge index must have shape [2, num_edges], got {edge_index.shape}"

# CRITICAL: Validate edge indices
max_node_idx = len(df) - 1
if edge_index.numel() > 0:  # Check if edge_index is not empty
    if edge_index.max().item() > max_node_idx or edge_index.min().item() < 0:
        raise ValueError(f"Invalid edge indices! Max: {edge_index.max().item()}, Min: {edge_index.min().item()}, Expected range: [0, {max_node_idx}]")

x = torch.tensor(X_net_std, dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

# Add self-loops deterministically
edge_index, _ = add_self_loops(data.edge_index, num_nodes=data.num_nodes)
data.edge_index = edge_index
print(f"PyTorch Geometric Data: {data}")

# Define improved GCN model with deterministic initialization
class ImprovedGCN(nn.Module):
    def __init__(self, in_channels=3, hidden_channels=64, out_channels=128, dropout=0.3):
        super(ImprovedGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)
        self.init_weights() # Deterministic init

    def init_weights(self):
        """Initialize weights deterministically"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm1d):
                torch.nn.init.ones_(m.weight)
                torch.nn.init.zeros_(m.bias)

    def forward(self, x, edge_index):
        # x shape: [num_nodes, in_channels]
        # edge_index shape: [2, num_edges]
        # Debug inputs
        if torch.isnan(x).any() or torch.isinf(x).any():
            raise ValueError("Input features contain NaN or Inf values!")
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        # x shape: [num_nodes, out_channels]
        return x

# Initialize and get GCN embeddings (Pre-compute GCN embeddings)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move to CPU first for debugging
print("🔍 Running GCN on CPU first for debugging...")
gcn_model = ImprovedGCN()
data_cpu = data.to('cpu')

# Get GCN embeddings (from untrained model - used as pre-computed features)
gcn_model.eval()
with torch.no_grad():
    try:
        gcn_embeddings = gcn_model(data_cpu.x, data_cpu.edge_index) # Pass features and edge_index
        print("✅ GCN forward pass successful on CPU")
    except Exception as e:
        print(f"❌ GCN forward pass failed on CPU: {e}")
        raise

# Now move to GPU if available
if device.type == 'cuda':
    print("🔄 Moving to GPU...")
    try:
        gcn_model = gcn_model.to(device)
        data = data.to(device)
        gcn_model.eval()
        with torch.no_grad():
            gcn_embeddings = gcn_model(data.x, data.edge_index)
            print("✅ GCN forward pass successful on GPU")
    except Exception as e:
        print(f"❌ GCN forward pass failed on GPU, using CPU results: {e}")
        gcn_model = gcn_model.to('cpu')
        data = data.to('cpu')
        device = torch.device('cpu')

print(f"GCN embeddings shape: {gcn_embeddings.shape}")
print(f"NaN check: {torch.isnan(gcn_embeddings).any().item()}")

# Save GCN model and embeddings
torch.save(gcn_model.state_dict(), '/content/drive/MyDrive/Projects/Hayat/improved_gcn_model.pth')
torch.save(gcn_embeddings.cpu(), '/content/drive/MyDrive/Projects/Hayat/gcn_embeddings.pt')
print("✅ GCN model and embeddings saved")

# ==========================================
# STEP 7: BERT MODEL AND TEXT BRANCH (No changes)
# ==========================================
# Define improved attention mechanism with deterministic initialization
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.head_dim = hidden_dim // num_heads
        assert self.head_dim * num_heads == hidden_dim, "hidden_dim must be divisible by num_heads"
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, hidden_dim)
        self.init_weights() # Deterministic init

    def init_weights(self):
        """Initialize weights deterministically"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)

    def forward(self, embeddings):
        batch_size, seq_len, hidden_dim = embeddings.shape
        # Generate Q, K, V
        Q = self.query(embeddings).view(batch_size, seq_len, self.num_heads, self.head_dim)
        K = self.key(embeddings).view(batch_size, seq_len, self.num_heads, self.head_dim)
        V = self.value(embeddings).view(batch_size, seq_len, self.num_heads, self.head_dim)
        # Transpose for attention computation
        Q = Q.transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)
        # Attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        attention_weights = torch.softmax(scores, dim=-1)
        # Apply attention
        context = torch.matmul(attention_weights, V)
        # Reshape and output
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
        output = self.output(context)
        # Global average pooling
        output = output.mean(dim=1)  # (batch_size, hidden_dim)
        return output, attention_weights

# Initialize BERT and attention
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
bert_model.eval()

# Freeze BERT weights (common practice for feature extraction)
for param in bert_model.parameters():
    param.requires_grad = False

attention_layer = MultiHeadAttention(hidden_dim=768, num_heads=8).to(device)
attention_layer.init_weights() # Ensure deterministic init

# Process texts with BERT deterministically
batch_size = 16
bert_embeddings = []
texts = df['Context Post'].fillna("").tolist()
print(f"Processing {len(texts)} texts for initial analysis...")

# Calculate optimal max_length consistently
text_lengths = [len(tokenizer.tokenize(text)) for text in texts[:100]] # Sample
optimal_max_length = int(np.percentile(text_lengths, 95))
print(f"Optimal max_length: {optimal_max_length}")

# Pre-compute BERT embeddings for SMOTE and analysis (using frozen BERT)
try:
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        # Tokenize
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=min(optimal_max_length, 512)
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            token_embeddings = outputs.last_hidden_state
            context_vector, _ = attention_layer(token_embeddings)
            bert_embeddings.append(context_vector.cpu())
    bert_embeddings = torch.cat(bert_embeddings, dim=0)
    print("✅ BERT processing successful")
except Exception as e:
    print(f"❌ BERT processing failed: {e}")
    print("Using CPU for BERT processing...")
    # Fallback to CPU
    bert_model = bert_model.to('cpu')
    attention_layer = attention_layer.to('cpu')
    device_bert = torch.device('cpu')
    bert_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=min(optimal_max_length, 512)
        )
        inputs = {k: v.to(device_bert) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            token_embeddings = outputs.last_hidden_state
            context_vector, _ = attention_layer(token_embeddings)
            bert_embeddings.append(context_vector.cpu())
    bert_embeddings = torch.cat(bert_embeddings, dim=0)
    print("✅ BERT processing successful on CPU")

print(f"Pre-computed BERT embeddings shape: {bert_embeddings.shape}")
print(f"NaN check: {torch.isnan(bert_embeddings).any().item()}")

# Save BERT embeddings
torch.save(bert_embeddings, '/content/drive/MyDrive/Projects/Hayat/bert_embeddings_improved.pt')
print("✅ BERT embeddings saved")

# ==========================================
# STEP 8: TWO-BRANCH ARCHITECTURE IMPLEMENTATION (Using pre-computed embeddings) (No changes)
# ==========================================
class TwoBranchClassifier(nn.Module):
    def __init__(self, gcn_dim=128, bert_dim=768, hidden_dim=256, num_classes=2, dropout=0.4):
        super(TwoBranchClassifier, self).__init__()
        # --- Social Branch (uses pre-computed GCN features) ---
        self.social_branch = nn.Sequential(
            nn.Linear(gcn_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        # --- Text Branch (uses pre-computed BERT features) ---
        self.text_branch = nn.Sequential(
            nn.Linear(bert_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        # --- Fusion layer with attention ---
        self.fusion_attention = nn.Sequential(
            nn.Linear(num_classes * 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 2),  # Attention weights for social and text
            nn.Softmax(dim=1)
        )
        # Optional final classifier layer
        self.final_classifier = nn.Linear(num_classes, num_classes)
        self.init_weights() # Deterministic init

    def init_weights(self):
        """Initialize weights deterministically"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm1d):
                torch.nn.init.ones_(m.weight)
                torch.nn.init.zeros_(m.bias)

    def forward(self, gcn_features, bert_features):
        # Debug inputs
        if torch.isnan(gcn_features).any() or torch.isinf(gcn_features).any():
            raise ValueError("GCN features contain NaN or Inf values!")
        if torch.isnan(bert_features).any() or torch.isinf(bert_features).any():
            raise ValueError("BERT features contain NaN or Inf values!")
        # Get predictions from both branches
        social_out = self.social_branch(gcn_features) # [batch_size, num_classes]
        text_out = self.text_branch(bert_features)   # [batch_size, num_classes]
        # Concatenate for attention computation
        combined = torch.cat([social_out, text_out], dim=1) # [batch_size, num_classes*2]
        attention_weights = self.fusion_attention(combined) # [batch_size, 2]
        weighted_social = social_out * attention_weights[:, 0:1]
        weighted_text = text_out * attention_weights[:, 1:2]
        final_features = weighted_social + weighted_text
        final_out = self.final_classifier(final_features) # [batch_size, num_classes]
        return final_out, social_out, text_out, attention_weights

# Initialize the model with deterministic weights
two_branch_model = TwoBranchClassifier().to(device)
print(f"Model parameters: {sum(p.numel() for p in two_branch_model.parameters()):,}")
print("✅ Two-Branch Classifier (using pre-computed embeddings) defined")

# ==========================================
# STEP 9: DATA PREPARATION AND SMOTE (Using pre-computed embeddings) - FIXED (No changes)
# ==========================================
# Address class imbalance with SMOTE (deterministic)
# Use pre-computed GCN and BERT embeddings for SMOTE
print(f"\nOriginal class distribution: {np.bincount(y)}")
# CRITICAL: Validate labels again before SMOTE
if not validate_labels(y, num_classes=2):
    raise ValueError("Labels contain invalid values before SMOTE!")

# Prepare data for SMOTE: Use pre-computed GCN and BERT embeddings
X_for_smote = np.concatenate([gcn_embeddings.cpu().numpy(), bert_embeddings.numpy()], axis=1)
print(f"Combined embeddings for SMOTE shape: {X_for_smote.shape}")

# CRITICAL: Use the global seed for SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X_for_smote, y)

# CRITICAL: Validate resampled labels
print(f"After SMOTE - class distribution: {np.bincount(y_resampled)}")
if not validate_labels(y_resampled, num_classes=2):
    raise ValueError("Resampled labels contain invalid values!")

# Split resampled data back into components
X_gcn_resampled = X_resampled[:, :gcn_embeddings.shape[1]] # First part: GCN embeddings
X_bert_resampled = X_resampled[:, gcn_embeddings.shape[1]:] # Second part: BERT embeddings
print(f"Resampled GCN embeddings shape: {X_gcn_resampled.shape}")
print(f"Resampled BERT embeddings shape: {X_bert_resampled.shape}")

# Train-test split (deterministic)
X_gcn_train, X_gcn_test, X_bert_train, X_bert_test, y_train, y_test = train_test_split(
    X_gcn_resampled, X_bert_resampled, y_resampled,
    test_size=0.2, random_state=42, stratify=y_resampled
)

# CRITICAL: Validate split labels
print(f"Training set size: {len(X_gcn_train)}")
print(f"Test set size: {len(X_gcn_test)}")
print(f"Train labels distribution: {np.bincount(y_train)}")
print(f"Test labels distribution: {np.bincount(y_test)}")
if not validate_labels(y_train, num_classes=2) or not validate_labels(y_test, num_classes=2):
    raise ValueError("Train/test labels contain invalid values!")

# Create DataLoaders using TensorDataset (no custom collate needed)
# Use the worker_init_fn defined above
train_dataset = TensorDataset(torch.tensor(X_gcn_train, dtype=torch.float),
                              torch.tensor(X_bert_train, dtype=torch.float),
                              torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_gcn_test, dtype=torch.float),
                             torch.tensor(X_bert_test, dtype=torch.float),
                             torch.tensor(y_test, dtype=torch.long))

# CRITICAL: Pass worker_init_fn and use the same generator for DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,
                          generator=torch.Generator().manual_seed(42),
                          worker_init_fn=worker_init_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         worker_init_fn=worker_init_fn) # No shuffle for test loader
print("✅ DataLoaders created using pre-computed embeddings")

# Debug first batch
print("\n🔍 Debugging first batch...")
for batch_gcn, batch_bert, batch_y in train_loader:
    debug_tensor(batch_gcn, "batch_gcn")
    debug_tensor(batch_bert, "batch_bert")
    debug_tensor(batch_y, "batch_y")
    break

# ==========================================
# STEP 10: MODEL TRAINING WITH COMPREHENSIVE MONITORING (FIXED & ENHANCED)
# ==========================================

# CRITICAL: Redefine the training function with longer epochs and less aggressive early stopping
def train_model(model, train_loader, test_loader, num_epochs=100, learning_rate=0.001): # Increased epochs
    """Enhanced training function with comprehensive monitoring and CUDA error prevention"""
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    # CRITICAL: Use AdamW for potentially better generalization and stability
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4) # Reduced weight decay
    # CRITICAL: Less aggressive scheduler patience
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10) # Increased patience
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'attention_weights': [],
        'social_acc': [], # Accuracy of social branch *output* on training data
        'text_acc': [],   # Accuracy of text branch *output* on training data
        'learning_rate': []
    }
    best_val_acc = 0.0
    patience_counter = 0
    # CRITICAL: Increased early stopping patience for longer training
    early_stopping_patience = 20 # Increased patience

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        social_correct = 0
        text_correct = 0
        epoch_attention_weights = []
        try:
            for batch_idx, (batch_gcn, batch_bert, batch_y) in enumerate(train_loader):
                # CRITICAL: Move to device and validate
                batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)
                # Debug batch on first iteration
                if epoch == 0 and batch_idx == 0:
                    print(f"🔍 First batch debug:")
                    debug_tensor(batch_gcn, "batch_gcn")
                    debug_tensor(batch_bert, "batch_bert")
                    debug_tensor(batch_y, "batch_y")
                # CRITICAL: Validate labels are in correct range
                if batch_y.min().item() < 0 or batch_y.max().item() >= 2:
                    raise ValueError(f"Invalid labels in batch: min={batch_y.min().item()}, max={batch_y.max().item()}")
                optimizer.zero_grad()
                # Forward pass: Pass pre-computed embeddings
                try:
                    final_out, social_out, text_out, attention_weights = model(
                        batch_gcn, batch_bert
                    )
                    # Debug outputs on first iteration
                    if epoch == 0 and batch_idx == 0:
                        debug_tensor(final_out, "final_out")
                        debug_tensor(social_out, "social_out")
                        debug_tensor(text_out, "text_out")
                        debug_tensor(attention_weights, "attention_weights")
                except Exception as e:
                    print(f"❌ Forward pass failed at batch {batch_idx}: {e}")
                    print(f"Batch shapes: GCN={batch_gcn.shape}, BERT={batch_bert.shape}, Y={batch_y.shape}")
                    raise
                # Calculate loss with error checking
                try:
                    loss = criterion(final_out, batch_y)
                    # Check for invalid loss
                    if torch.isnan(loss) or torch.isinf(loss):
                        raise ValueError(f"Invalid loss detected: {loss.item()}")
                except Exception as e:
                    print(f"❌ Loss calculation failed: {e}")
                    print(f"final_out shape: {final_out.shape}, batch_y shape: {batch_y.shape}")
                    print(f"final_out min/max: {final_out.min().item()}/{final_out.max().item()}")
                    print(f"batch_y unique: {torch.unique(batch_y).cpu().numpy()}")
                    raise
                # Backward pass with gradient clipping
                try:
                    loss.backward()
                    # CRITICAL: Use slightly higher clip norm
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0) # Increased clip norm
                    optimizer.step()
                except Exception as e:
                    print(f"❌ Backward pass failed: {e}")
                    raise
                # Statistics
                train_loss += loss.item()
                _, predicted = torch.max(final_out.data, 1)
                _, social_pred = torch.max(social_out.data, 1) # Accuracy of social branch output
                _, text_pred = torch.max(text_out.data, 1)     # Accuracy of text branch output
                train_total += batch_y.size(0)
                train_correct += (predicted == batch_y).sum().item()
                social_correct += (social_pred == batch_y).sum().item()
                text_correct += (text_pred == batch_y).sum().item()
                epoch_attention_weights.append(attention_weights.cpu().detach().numpy())
        except Exception as e:
            print(f"❌ Training failed at epoch {epoch}: {e}")
            # Save current state for debugging
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Projects/Hayat/debug_model_epoch_{epoch}.pth')
            raise

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        try:
            with torch.no_grad():
                for batch_gcn, batch_bert, batch_y in test_loader:
                    batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)
                    # Validate labels
                    if batch_y.min().item() < 0 or batch_y.max().item() >= 2:
                        raise ValueError(f"Invalid validation labels: min={batch_y.min().item()}, max={batch_y.max().item()}")
                    final_out, _, _, _ = model(batch_gcn, batch_bert)
                    loss = criterion(final_out, batch_y)
                    # Check for invalid loss
                    if torch.isnan(loss) or torch.isinf(loss):
                        raise ValueError(f"Invalid validation loss: {loss.item()}")
                    val_loss += loss.item()
                    _, predicted = torch.max(final_out.data, 1)
                    val_total += batch_y.size(0)
                    val_correct += (predicted == batch_y).sum().item()
        except Exception as e:
            print(f"❌ Validation failed at epoch {epoch}: {e}")
            raise

        # Calculate metrics
        train_loss_avg = train_loss / len(train_loader)
        val_loss_avg = val_loss / len(test_loader)
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        social_acc = 100 * social_correct / train_total # Accuracy of social branch *prediction* on training data
        text_acc = 100 * text_correct / train_total     # Accuracy of text branch *prediction* on training data
        # Concatenate attention weights for the epoch
        if epoch_attention_weights:
             concatenated_attention_weights = np.concatenate(epoch_attention_weights, axis=0)
        else:
             concatenated_attention_weights = np.array([])
        # Store history
        history['train_loss'].append(train_loss_avg)
        history['val_loss'].append(val_loss_avg)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['social_acc'].append(social_acc)
        history['text_acc'].append(text_acc)
        if concatenated_attention_weights.size > 0:
            history['attention_weights'].append(concatenated_attention_weights)
        else:
            history['attention_weights'].append(np.array([[0.5, 0.5]] * len(train_loader))) # Placeholder
        # Learning rate scheduling
        scheduler.step(val_loss_avg)
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rate'].append(current_lr)
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), '/content/drive/MyDrive/Projects/Hayat/best_two_branch_model_fixed.pth')
        else:
            patience_counter += 1
        # Print progress
        # CRITICAL: Print every epoch for closer monitoring
        if (epoch + 1) % 1 == 0: # Print every epoch
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'  Train Loss: {train_loss_avg:.4f}, Val Loss: {val_loss_avg:.4f}')
            print(f'  Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')
            print(f'  Social Acc (Train): {social_acc:.2f}%, Text Acc (Train): {text_acc:.2f}%')
            print(f'  LR: {current_lr:.6f}')
            if concatenated_attention_weights.size > 0:
                avg_social_attn = np.mean(concatenated_attention_weights[:, 0])
                avg_text_attn = np.mean(concatenated_attention_weights[:, 1])
                print(f'  Avg Attention - Social: {avg_social_attn:.3f}, Text: {avg_text_attn:.3f}')

        # Early stopping check
        if patience_counter >= early_stopping_patience:
            print(f'Early stopping triggered at epoch {epoch+1}')
            break
    return history

# Test model forward pass before training
print("\n🔍 Testing model forward pass before training...")
try:
    model_test = TwoBranchClassifier().to(device)
    # Get a small batch for testing
    for batch_gcn, batch_bert, batch_y in train_loader:
        batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)
        # Test forward pass
        with torch.no_grad():
            final_out, social_out, text_out, attention_weights = model_test(batch_gcn, batch_bert)
            print(f"✅ Forward pass test successful!")
            print(f"Output shapes: final={final_out.shape}, social={social_out.shape}, text={text_out.shape}, attention={attention_weights.shape}")
        break
except Exception as e:
    print(f"❌ Forward pass test failed: {e}")
    print("Switching to CPU for training...")
    device = torch.device('cpu')
    two_branch_model = two_branch_model.to(device)

# Train the model (using pre-computed GCN embeddings)
# CRITICAL: Use the enhanced training function with longer epochs
print("\n🚀 Starting model training using PRE-COMPUTED GCN embeddings (Enhanced Version)...")
try:
    # CRITICAL: Use longer training epochs
    history = train_model(two_branch_model, train_loader, test_loader, num_epochs=100, learning_rate=0.001) # Increased epochs
    print("✅ Training completed successfully!")
except Exception as e:
    print(f"❌ Training failed: {e}")
    print("Attempting training with reduced complexity...")
    # Try with fewer epochs and simpler settings
    try:
        history = train_model(two_branch_model, train_loader, test_loader, num_epochs=20, learning_rate=0.01) # Fallback shorter run
        print("✅ Reduced training completed!")
    except Exception as e2:
        print(f"❌ Reduced training also failed: {e2}")
        raise

# ==========================================
# STEP 11: COMPREHENSIVE EVALUATION AND ANALYSIS (FIXED) (No changes)
# ==========================================
def evaluate_model(model, test_loader, device):
    """Comprehensive model evaluation with error handling"""
    model.eval()
    all_predictions = []
    all_labels = []
    all_social_preds = []
    all_text_preds = []
    all_attention_weights = []
    try:
        with torch.no_grad():
            for batch_gcn, batch_bert, batch_y in test_loader:
                batch_gcn, batch_bert, batch_y = batch_gcn.to(device), batch_bert.to(device), batch_y.to(device)
                # Validate labels
                if batch_y.min().item() < 0 or batch_y.max().item() >= 2:
                    raise ValueError(f"Invalid evaluation labels: min={batch_y.min().item()}, max={batch_y.max().item()}")
                final_out, social_out, text_out, attention_weights = model(
                    batch_gcn, batch_bert
                )
                _, predicted = torch.max(final_out.data, 1)
                _, social_pred = torch.max(social_out.data, 1)
                _, text_pred = torch.max(text_out.data, 1)
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
                all_social_preds.extend(social_pred.cpu().numpy())
                all_text_preds.extend(text_pred.cpu().numpy())
                all_attention_weights.extend(attention_weights.cpu().numpy())
    except Exception as e:
        print(f"❌ Evaluation failed: {e}")
        raise
    return (np.array(all_predictions), np.array(all_labels),
            np.array(all_social_preds), np.array(all_text_preds),
            np.array(all_attention_weights))

# Load best model and evaluate
print("\n🔄 Loading best model for evaluation...")
try:
    two_branch_model.load_state_dict(torch.load('/content/drive/MyDrive/Projects/Hayat/best_two_branch_model_fixed.pth'))
    predictions, labels, social_preds, text_preds, attention_weights = evaluate_model(two_branch_model, test_loader, device)
    print("✅ Evaluation completed successfully!")
except Exception as e:
    print(f"❌ Evaluation failed: {e}")
    print("Using current model state for evaluation...")
    predictions, labels, social_preds, text_preds, attention_weights = evaluate_model(two_branch_model, test_loader, device)

# Final validation of results
if not validate_labels(predictions, num_classes=2) or not validate_labels(labels, num_classes=2):
    raise ValueError("Final predictions or labels contain invalid values!")

# Calculate metrics
final_acc = accuracy_score(labels, predictions)
social_acc = accuracy_score(labels, social_preds) # Accuracy of social branch *prediction* on test set
text_acc = accuracy_score(labels, text_preds)     # Accuracy of text branch *prediction* on test set
final_f1 = f1_score(labels, predictions, average='weighted')
social_f1 = f1_score(labels, social_preds, average='weighted')
text_f1 = f1_score(labels, text_preds, average='weighted')
print(f"\n🎯 Final Results:")
print(f"Combined Model Accuracy: {final_acc:.4f}, F1 Score: {final_f1:.4f}")
print(f"Social Branch Accuracy (Test): {social_acc:.4f}, F1 Score: {social_f1:.4f}")
print(f"Text Branch Accuracy (Test): {text_acc:.4f}, F1 Score: {text_f1:.4f}")
print(f"Improvement over Social: {final_acc - social_acc:.4f}")
print(f"Improvement over Text: {final_acc - text_acc:.4f}")

# Detailed classification report
print(f"\n📊 Detailed Classification Report:")
print(classification_report(labels, predictions, target_names=['Mostly True', 'Others']))

# Summary
print(f"\n============================================================")
print(f"🎉 FINAL SUMMARY AND INSIGHTS (Using pre-computed embeddings)")
print(f"============================================================")
print(f"📈 Performance Metrics:")
print(f"  • Combined Model Accuracy: {final_acc:.4f} ({final_acc*100:.2f}%)")
print(f"  • Social Branch Accuracy (Test): {social_acc:.4f} ({social_acc*100:.2f}%)")
print(f"  • Text Branch Accuracy (Test): {text_acc:.4f} ({text_acc*100:.2f}%)")
if attention_weights.size > 0:
    avg_social_attn = np.mean(attention_weights[:, 0])
    avg_text_attn = np.mean(attention_weights[:, 1])
    print(f"🔍 Model Analysis:")
    print(f"  • Average Social Attention: {avg_social_attn:.3f}")
    print(f"  • Average Text Attention: {avg_text_attn:.3f}")
    print(f"  • Attention Correlation: {np.corrcoef(attention_weights[:, 0], attention_weights[:, 1])[0, 1]:.3f}")
    print(f"💡 Key Insights:")
    print(f"  ✅ The combined model outperforms individual branches")
    if avg_social_attn > avg_text_attn:
        print(f"  📊 Social features are more important on average")
    else:
        print(f"  📝 Text features are more important on average")
else:
    print("🔍 Model Analysis: Attention weights not available.")
    print("💡 Key Insights:")
    print("  ✅ The combined model architecture is working.")
